# Convolutional Neural Netowrks 

In [6]:
import os
import numpy as np
import tensorflow as tf

In [7]:
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
mnist_dataset = input_data.read_data_sets("../data/mnist/", one_hot=True)

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [9]:
NUM_CLASSES = mnist_dataset.train.labels.shape[1]# Number of category

IMAGE_H = 28 # hight (pixel) of image
IMAGE_W = 28 # width (pixel) of image
IMAGE_C = 1  # channel of iamge

CONV1_C = 32 # First convolutional layer's channel size
CONV1_F = 5  # First convolutional layer's kernel szie

CONV2_C = 64 # Second convolutional layer's channel size
CONV2_F = 5  # Second convolutional layer's kernel size

FC_SIZE = 512 # Fully connected layer's number of node


BATH_SIZE = 32
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0001
TRAINING_STEPS = 1000
MOVING_AVERAGE_DECAY = 0.99

MODE_SAVE_PATH = "model/"
MODE_NAME = "minst_model.ckpt"

## Define model structure

In [10]:
def inference(images, regularizer, is_train=False):
    
    with tf.variable_scope("layer1-conv1"):
        # Define convolution weights and biases
        conv1_weights = tf.get_variable(
            name="weight", 
            shape=[CONV1_F, CONV1_F, IMAGE_C, CONV1_C], 
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv1_biases = tf.get_variable(
            name="bias", 
            shape=[CONV1_C],
            initializer=tf.constant_initializer(0.0)
        )
        
        # input is a tensor that shape is [batch, in_height, in_width, in_channels]
        # filter is a tensor that shape is [filter_height, filter_width, in_channels, out_channels]
        # strides is a 1-D tensor of length 4. The stride of the sliding window for each dimension of input
        # padding is a string from: "SAME", "VALID". The type of padding algorithm to use.
        conv1 = tf.nn.conv2d(
            input=images,
            filter=conv1_weights,
            strides=[1, 1, 1, 1],
            padding="SAME"
        )
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))
    
    with tf.name_scope("layer2-pool1"):
        # value is a 4-D Tensor with shape [batch, height, width, channels] and type tf.float32.
        # ksize is a list of ints that has length >= 4. The size of the window for each dimension of the input tensor.
        # strides is a list of ints that has length >= 4. The stride of the sliding window for each dimension of the input tensor.
        # padding is a string from: "SAME", "VALID". The type of padding algorithm to use.
        pool1 = tf.nn.max_pool(
            value=relu1, 
            ksize=[1, 2, 2, 1], 
            strides=[1, 2, 2, 1], 
            padding="SAME"
        )
        
    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            name="weight",
            shape=[CONV2_F, CONV2_F, CONV1_C, CONV2_C],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv2_biases = tf.get_variable(
            name="bias",
            shape=[CONV2_C],
            initializer=tf.constant_initializer(0.0)
        )
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding="SAME")
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(
            value=relu2,
            ksize=[1, 2, 2, 1],
            strides=[1, 2, 2, 1],
            padding="SAME"
        )
        
    # Resahpe layer4-pool2 output size to fit layer5-fc1 input size
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    reshaped = tf.reshape(pool2, [pool_shape[0], nodes])
    
    with tf.variable_scope("layer5-fc1"):
        fc1_weights = tf.get_variable(
            name="weights",
            shape=[nodes, FC_SIZE],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        if regularizer is not None:
            tf.add_to_collection("losses", regularizer(fc1_weights))
        fc1_biases = tf.get_variable(
            name="bias",
            shape=[FC_SIZE],
            initializer=tf.constant_initializer(0.1)
        )
        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if is_train:
            fc1 = tf.nn.dropout(fc1, keep_prob=0.5)
            
    with tf.variable_scope("layer6-fc2"):
        fc2_weights = tf.get_variable(
            name="weights",
            shape=[FC_SIZE, NUM_CLASSES],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        if regularizer is not None:
            tf.add_to_collection("losses", regularizer(fc2_weights))
        fc2_biases = tf.get_variable(
            name="bias",
            shape=[NUM_CLASSES],
            initializer=tf.constant_initializer(0.1)
        )
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases
    
    return logit

In [ ]:
def train(mnist_dataset):
    
    # Define input data (train_X, trian_Y) placeholder
    input_X = tf.placeholder(
        tf.float32, 
        shape=(BATH_SIZE, IMAGE_H, IMAGE_W, IMAGE_C),
        name="x-input")
    
    input_Y = tf.placeholder(
        tf.float32,
        shape=(None, NUM_CLASSES),
        name="y-input")
    
    # Define inference
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    predicted_Y = inference(input_X, regularizer, is_train=True)
    
    global_step = tf.Variable(0, trainable=False)
    
    # Define loss, learning rate
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=predicted_Y, 
        labels=tf.argmax(input_Y, 1)
    )
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection("losses"))
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist_dataset.train.num_examples/BATH_SIZE, LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step)
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name="train")
        
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i  in range(TRAINING_STEPS):
            X, Y = mnist_dataset.train.next_batch(BATH_SIZE)
            reshaped_X = np.reshape(X, (BATH_SIZE, IMAGE_H, IMAGE_W, IMAGE_C))
            _, loss_value, step = sess.run([train_op, loss, global_step], 
                                           feed_dict={input_X: reshaped_X, input_Y: Y})
            
            if i % 1000 == 0:
                print("After {} training steps, loos on training batch is {}".format(step, loss_value))
                saver.save(sess, os.path.join(MODE_SAVE_PATH, MODE_NAME))
                
    
    
    

In [ ]:
train(mnist_dataset)

In [ ]:
import time
def evaluate(mnist_dataset):
    
    with tf.Graph().as_default() as g:
        
        input_shape = (mnist_dataset.validation.num_examples, IMAGE_H, IMAGE_W, IMAGE_C)
        input_X = tf.placeholder(
            tf.float32, 
            shape=input_shape,
            name="x-input")
        input_Y = tf.placeholder(
            tf.float32,
            shape=(None, NUM_CLASSES),
            name="y-input"
        )
        
        predict_Y = inference(input_X, None, is_train=False)
        
        validate_feed = {
            input_X: np.reshape(mnist_dataset.validation.images, input_shape),
            input_Y: mnist_dataset.validation.labels
        }
        
        correct_prediction = tf.equal(tf.argmax(predict_Y, 1), tf.argmax(input_Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variable_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variable_to_restore)
        
        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODE_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split("/")[-1].split("-")[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print("After {} training steps, validation accuracty {}".format(global_step, accuracy_score))
                else:
                    print("No checkpoint file found")
                    return
            time.sleep(10)
        

In [ ]:
evaluate(mnist_dataset)

In [24]:
def predict(mnist_data):
    
    with tf.Graph().as_default() as g:
        
        input_shape = (1, IMAGE_H, IMAGE_W, IMAGE_C)
        input_X = tf.placeholder(
            tf.float32, 
            shape=input_shape,
            name="x-input")
        
        predict_Y = inference(input_X, None, is_train=False)
        probabilities = tf.nn.softmax(predict_Y)
        
        validate_feed = {
            input_X: mnist_data
            # input_Y: mnist_dataset.validation.labels
        }
        
        # correct_prediction = tf.equal(tf.argmax(predict_Y, 1), tf.argmax(input_Y, 1))
        # accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variable_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variable_to_restore)
        
        # while True:
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(MODE_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                global_step = ckpt.model_checkpoint_path.split("/")[-1].split("-")[-1]
                print(sess.run(probabilities, feed_dict=validate_feed))
                # print(sess.run(predict_Y, feed_dict=validate_feed))
            else:
                print("No checkpoint file found")

                    
        

In [23]:
input_image = np.reshape(mnist_dataset.validation.images[1], (1, IMAGE_H, IMAGE_W, IMAGE_C))
predict(input_image)

INFO:tensorflow:Restoring parameters from model/minst_model.ckpt
[[ 2.698931   -1.0290753   1.5153164   1.4677272  -1.2578548  -0.47523215
   4.097725   -0.56527877 -2.6150303  -3.3029695 ]]
